## Paso 1: Importar librerías necesarias

Utilizamos `heapq` para implementar una cola de prioridad (min-heap) que nos permitirá construir el árbol de Huffman de manera eficiente.

In [ ]:
import heapq

## Paso 2: Definir el texto a codificar

Para este ejemplo, usaremos un texto de prueba. En la práctica, podrías leer el texto desde un archivo o desde la entrada del usuario.

In [ ]:
# Texto de ejemplo para codificación
texto = "ejemplo de texto para codificacion huffman"
print(f"Texto a codificar: '{texto}'")
print(f"Longitud del texto: {len(texto)} caracteres")

## Paso 3: Calcular frecuencias de cada carácter

El primer paso del algoritmo es contar cuántas veces aparece cada carácter en el texto. Esto nos permitirá asignar códigos más cortos a los caracteres más frecuentes.

In [ ]:
# Calcular frecuencias de cada carácter
frecuencias = {}

for letra in texto:
    if letra in frecuencias:
        frecuencias[letra] += 1
    else:
        frecuencias[letra] = 1

print("Frecuencias de cada carácter:")
print("-" * 30)
for letra, freq in sorted(frecuencias.items(), key=lambda x: x[1], reverse=True):
    print(f"  '{letra}': {freq} veces")

## Paso 4: Definir la estructura del Nodo

Cada nodo del árbol de Huffman contiene:
- **letra**: El carácter (solo en nodos hoja)
- **freq**: La frecuencia del carácter o suma de frecuencias de hijos
- **izquierda**: Referencia al hijo izquierdo (código '0')
- **derecha**: Referencia al hijo derecho (código '1')

El método `__lt__` permite comparar nodos por frecuencia, necesario para el heap.

In [ ]:
class Nodo:
    def __init__(self, letra, freq):
        self.letra = letra      # Carácter (None para nodos internos)
        self.freq = freq        # Frecuencia
        self.izquierda = None   # Hijo izquierdo (código '0')
        self.derecha = None     # Hijo derecho (código '1')

    def __lt__(self, otro):
        """Comparación necesaria para el heap (min-heap por frecuencia)"""
        return self.freq < otro.freq
    
    def __repr__(self):
        if self.letra:
            return f"Nodo('{self.letra}', {self.freq})"
        return f"Nodo(interno, {self.freq})"

print("Clase Nodo definida correctamente ✓")

## Paso 5: Construir el Árbol de Huffman

El algoritmo para construir el árbol es:

1. Crear un nodo hoja para cada carácter con su frecuencia
2. Insertar todos los nodos en un min-heap (cola de prioridad)
3. Mientras haya más de un nodo en el heap:
   - Extraer los dos nodos con menor frecuencia
   - Crear un nuevo nodo interno con estos dos como hijos
   - La frecuencia del nuevo nodo es la suma de las frecuencias de sus hijos
   - Insertar el nuevo nodo en el heap
4. El nodo restante es la raíz del árbol

In [ ]:
# Crear nodos hoja para cada carácter
heap = [Nodo(letra, freq) for letra, freq in frecuencias.items()]
print(f"Nodos iniciales ({len(heap)} nodos hoja):")
for nodo in sorted(heap, key=lambda x: x.freq):
    print(f"  {nodo}")

# Convertir la lista en un min-heap
heapq.heapify(heap)
print(f"\nHeap creado con {len(heap)} elementos")

In [ ]:
# Construir el árbol de Huffman
paso = 1
print("Construcción del árbol de Huffman:")
print("=" * 50)

while len(heap) > 1:
    # Extraer los dos nodos con menor frecuencia
    nodo1 = heapq.heappop(heap)
    nodo2 = heapq.heappop(heap)
    
    # Crear nuevo nodo interno
    nuevo_nodo = Nodo(None, nodo1.freq + nodo2.freq)
    nuevo_nodo.izquierda = nodo1
    nuevo_nodo.derecha = nodo2
    
    print(f"Paso {paso}: Combinar {nodo1} + {nodo2} = {nuevo_nodo}")
    
    # Insertar el nuevo nodo en el heap
    heapq.heappush(heap, nuevo_nodo)
    paso += 1

print("=" * 50)
print(f"\n¡Árbol construido! Raíz con frecuencia total: {heap[0].freq}")

## Paso 6: Generar los códigos de Huffman

Para generar los códigos, recorremos el árbol desde la raíz hasta cada hoja:
- Al ir por la rama **izquierda**, añadimos un **'0'**
- Al ir por la rama **derecha**, añadimos un **'1'**

El código de cada carácter es la secuencia de bits desde la raíz hasta su nodo hoja.

In [ ]:
codigos_huffman = {}

def generar_codigos(nodo, codigo=""):
    """Genera los códigos de Huffman recorriendo el árbol recursivamente."""
    if nodo is None:
        return
    
    # Si es un nodo hoja, guardamos el código
    if nodo.letra is not None:
        codigos_huffman[nodo.letra] = codigo
        return
    
    # Recorrer hijo izquierdo (añadir '0')
    generar_codigos(nodo.izquierda, codigo + "0")
    # Recorrer hijo derecho (añadir '1')
    generar_codigos(nodo.derecha, codigo + "1")

# Obtener la raíz y generar códigos
raiz = heap[0]
generar_codigos(raiz)

print("Códigos de Huffman generados:")
print("-" * 40)
for letra, codigo in sorted(codigos_huffman.items(), key=lambda x: len(x[1])):
    print(f"  '{letra}' -> {codigo} ({len(codigo)} bits)")

## Paso 7: Visualizar el Árbol de Huffman

Mostramos el árbol de forma visual para entender mejor su estructura. Los nodos internos muestran la suma de frecuencias entre corchetes `[freq]`, y los nodos hoja muestran el carácter y su frecuencia.

In [ ]:
def mostrar_arbol(nodo, prefijo="", es_ultimo=True):
    """Muestra el árbol de Huffman de forma visual."""
    if nodo is None:
        return
    
    # Determinar el conector
    conector = "└── " if es_ultimo else "├── "
    
    # Etiqueta del nodo
    if nodo.letra is not None:
        etiqueta = f"'{nodo.letra}' (freq={nodo.freq})"
    else:
        etiqueta = f"[{nodo.freq}]"
    
    print(prefijo + conector + etiqueta)
    
    # Nuevo prefijo para los hijos
    nuevo_prefijo = prefijo + ("    " if es_ultimo else "│   ")
    
    # Mostrar hijos
    hijos = []
    if nodo.izquierda:
        hijos.append((nodo.izquierda, "0"))
    if nodo.derecha:
        hijos.append((nodo.derecha, "1"))
    
    for i, (hijo, dir) in enumerate(hijos):
        es_ultimo_hijo = (i == len(hijos) - 1)
        mostrar_arbol(hijo, nuevo_prefijo, es_ultimo_hijo)

print("\n" + "="*60)
print("ÁRBOL DE HUFFMAN")
print("="*60)
print("Raíz")
mostrar_arbol(raiz, "", True)
print("="*60)

## Paso 8: Codificar el texto

Ahora que tenemos los códigos, podemos codificar el texto original reemplazando cada carácter por su código de Huffman.

In [ ]:
# Codificar el texto
texto_codificado = "".join(codigos_huffman[letra] for letra in texto)

print("Texto original:")
print(f"  '{texto}'")
print(f"\nTexto codificado (primeros 100 bits):")
print(f"  {texto_codificado[:100]}..." if len(texto_codificado) > 100 else f"  {texto_codificado}")
print(f"\nLongitud total del texto codificado: {len(texto_codificado)} bits")

## Paso 9: Análisis de compresión

Comparamos el tamaño del texto original (usando codificación ASCII de 8 bits por carácter) con el tamaño del texto codificado con Huffman.

In [ ]:
# Calcular bits
bits_originales = len(texto) * 8  # ASCII usa 8 bits por carácter
bits_codificados = sum(frecuencias[letra] * len(codigos_huffman[letra]) 
                       for letra in codigos_huffman)

# Calcular ratio de compresión
ratio_compresion = (1 - bits_codificados / bits_originales) * 100
bits_promedio = bits_codificados / len(texto)

print("ANÁLISIS DE COMPRESIÓN")
print("=" * 50)
print(f"Bits en texto original (ASCII):    {bits_originales} bits")
print(f"Bits en texto codificado (Huffman): {bits_codificados} bits")
print(f"Ahorro:                             {bits_originales - bits_codificados} bits")
print(f"Ratio de compresión:                {ratio_compresion:.2f}%")
print(f"Bits promedio por carácter:         {bits_promedio:.2f} bits")
print("=" * 50)

## Paso 10: Decodificación

Para decodificar, recorremos el árbol desde la raíz siguiendo los bits:
- **'0'** → ir a la izquierda
- **'1'** → ir a la derecha

Cuando llegamos a un nodo hoja, hemos encontrado un carácter.

In [ ]:
def decodificar(texto_codificado, raiz):
    """Decodifica un texto codificado con Huffman."""
    texto_decodificado = []
    nodo_actual = raiz
    
    for bit in texto_codificado:
        # Navegar por el árbol según el bit
        if bit == '0':
            nodo_actual = nodo_actual.izquierda
        else:
            nodo_actual = nodo_actual.derecha
        
        # Si llegamos a un nodo hoja, encontramos un carácter
        if nodo_actual.letra is not None:
            texto_decodificado.append(nodo_actual.letra)
            nodo_actual = raiz  # Volver a la raíz
    
    return ''.join(texto_decodificado)

## Resumen

### Pasos del algoritmo de Huffman:
1. **Calcular frecuencias** de cada símbolo en el texto
2. **Crear nodos hoja** para cada símbolo con su frecuencia
3. **Construir el árbol** combinando los nodos de menor frecuencia
4. **Generar códigos** recorriendo el árbol (0=izquierda, 1=derecha)
5. **Codificar** reemplazando cada símbolo por su código

### Ventajas:
- Compresión sin pérdida
- Códigos de longitud óptima
- Decodificación no ambigua (código prefijo)

### Taza de compresion

(1 - Bits Comprimidos/Bits Originales) x 100